# Модуль 4.2 — Эмбеддинги и ретриверы

**Цель:** создать векторное хранилище и настроить поиск релевантных фрагментов.

**Что сделаем:**
- установим библиотеки
- создадим эмбеддинги с кэшированием
- построим Chroma‑хранилище
- сделаем простой ретривер

In [ ]:
%pip -q install -U \
  langchain \
  langchain-community \
  langchain-openai \
  chromadb \
  pydantic==2.12.3 \
  requests==2.32.4

## Если Colab ругается на зависимости

Иногда в Colab установлены библиотеки с жёсткими версиями (например, `google-adk` и `opentelemetry`).
Если после установки появляются конфликты — просто зафиксируйте совместимые версии ниже.

In [ ]:
%pip -q install -U \
  opentelemetry-api==1.37.0 \
  opentelemetry-sdk==1.37.0 \
  opentelemetry-proto==1.37.0 \
  opentelemetry-exporter-otlp-proto-common==1.37.0

In [ ]:
%pip -q install -U \
  opentelemetry-exporter-otlp-proto-grpc==1.37.0

# На случай, если в окружении уже стоит более новая версия
def _force_pins():
    import sys, subprocess
    pkgs = [
        "opentelemetry-api==1.37.0",
        "opentelemetry-sdk==1.37.0",
        "opentelemetry-proto==1.37.0",
        "opentelemetry-exporter-otlp-proto-common==1.37.0",
        "opentelemetry-exporter-otlp-proto-grpc==1.37.0",
    ]
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "-U", "--force-reinstall"] + pkgs)

_force_pins()

## Настройка ключа и base URL

Для AITunnel укажите `OPENAI_API_KEY` и `OPENAI_BASE_URL`. В Colab лучше хранить ключ в переменной окружения.

> Подсказка: можно задать ключ один раз в разделе **Secrets** (значок 🔑 слева в Colab), тогда он будет доступен во всех ячейках.

In [ ]:
import os
from getpass import getpass
from dotenv import load_dotenv

load_dotenv()

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass("Введите OPENAI_API_KEY: ")

if not os.environ.get("OPENAI_BASE_URL"):
    os.environ["OPENAI_BASE_URL"] = "https://api.aitunnel.ru/v1/"

## Эмбеддинги с кэшированием

Кэширование экономит время и деньги: уже рассчитанные эмбеддинги не пересчитываются.
Мы используем `CacheBackedEmbeddings` и локальное хранилище.

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain.storage import LocalFileStore
from langchain.embeddings import CacheBackedEmbeddings

base_url = os.environ.get("OPENAI_BASE_URL")

store = LocalFileStore("./cache/embeddings")
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",
    base_url=base_url,
)

cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    embeddings,
    store,
    namespace="openai",
)

## Подготовка папки `data`

Если папки нет, создадим её и положим пример файла, чтобы всё запускалось сразу.

In [ ]:
from pathlib import Path

data_dir = Path("./data")
data_dir.mkdir(parents=True, exist_ok=True)

sample_path = data_dir / "sample.txt"
if not sample_path.exists():
    sample_path.write_text(
        "LangChain помогает строить цепочки (chains), агенты и RAG‑системы.\n"
        "RAG улучшает ответы, добавляя знания из документов.\n"
        "Эмбеддинги позволяют искать похожие фрагменты текста.",
        encoding="utf-8",
    )

print("Файлы в data:", [p.name for p in data_dir.glob("*.txt")])

## Векторное хранилище и ретривер

Используем Chroma и получаем ретривер для поиска релевантных чанков.

In [ ]:
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = DirectoryLoader(
    "./data",
    glob="**/*.txt",
    loader_cls=TextLoader,
    show_progress=True,
)

documents = loader.load()

splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
)

chunks = splitter.split_documents(documents)

vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=cached_embedder,
    persist_directory="./db/chroma",
)

retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

query = "Что такое LangChain?"
results = retriever.get_relevant_documents(query)

for i, doc in enumerate(results, 1):
    print(f"\nРезультат {i}:")
    print(doc.page_content[:400])

## Практические рекомендации

- Кэшируйте эмбеддинги — это ускоряет эксперименты и снижает цену.
- Храните `persist_directory`, чтобы не терять векторную базу.
- Начните с `k=3..5` и подбирайте под ваш домен.
- При слабых результатах попробуйте увеличить `chunk_size` или изменить данные.